In [73]:
# streamlit run stream_test.py
import yfinance as yf
import streamlit as st
import datetime
import plotly.graph_objects as go
import plotly
import pandas as pd
import numpy as np
import requests_cache
session = requests_cache.CachedSession('yfinance.cache')
session.headers['User-agent'] = 'my-program/1.0'

In [57]:
def take_data(Company_tiker: str,date_of_buy='2021-04-28'):
    tickerSymbol = Company_tiker.upper()

    now_now = datetime.datetime.today()

    now_data = now_now.strftime('%Y-%m-%d')

    start_t = datetime.datetime.strptime(date_of_buy,"%Y-%m-%d")
    

    tickerData = yf.Ticker(tickerSymbol)

    tickerDf = tickerData.history(period='1d', start=start_t, end=now_data)

    return tickerDf

In [58]:
current_course = take_data('RUB=X').max()['Close']

In [59]:
def make_candle(df):
    fig = go.Figure(data=[go.Candlestick(x=df.index,
                                     open=df['Open'], high=df['High'],
                                     low=df['Low'], close=df['Close'])
                      ])
    return fig

In [60]:
def make_dict_from_company(basic_porfolio_dox: 'excel document'):
    dict_companies_data = {}
    for index,row in account.iterrows():
        dict_companies_data[row['tiker']] = take_data(row['tiker'],str(row['date_purchase'].strftime('%Y-%m-%d')))
        dict_companies_data[row['tiker']]['date'] = dict_companies_data[row['tiker']].index
    return dict_companies_data

In [61]:
from functools import reduce
def count_mean_measure_all_sec(dict_companies_data):
    ac = pd.DataFrame()
    frames_number = len(dict_companies_data)
    stock_measurets = ['Open', 'High', 'Low', 'Close', 'Volume', 'Dividends', 'Stock Splits']
    for measure in stock_measurets:
        ac[measure] = reduce(
            lambda x,y: dict_companies_data[x][measure]+dict_companies_data[y][measure] ,
        dict_companies_data.keys()
        )/frames_number
    return ac

In [62]:
account = pd.read_excel('my_account.xlsx')
account.date_purchase = pd.to_datetime(account.date_purchase, format="%m-%d-%Y")
account

,tiker,price_buy,lots_buy,date_purchase
0,STRA,93,350,2021-04-28
1,TNDM,94,110,2021-04-28


In [63]:
def porfolio_result(account=account):
    dictionary_info = {}
    dictionary_info['value_in_usd'] = round(account.result.sum())
    dictionary_info['value_in_rub'] =round(account.result.sum())*73
    dictionary_info['buy_sum_total'] = account['sum'].sum()
    dictionary_info['percent'] = dictionary_info['value_in_usd']/dictionary_info['buy_sum_total']
    dictionary_info['percent_display'] = f'{round(dictionary_info["percent"]*100,2)}%'

    return dictionary_info

In [64]:
dict_companies_data = make_dict_from_company(account)

In [65]:
def current_account_state():
    account['sum'] = account['price_buy'] * account['lots_buy']
    account['now_price'] = list(map(now_price,account.tiker))
    account['now_sum'] = round(account['now_price']*account['lots_buy'],0)
    account['result'] = account['now_sum'] - account['sum']
    account['percentage'] = account['result']/account['sum']
    
    return porfolio_result()

In [66]:
current_account_state()

{'value_in_usd': -7409,
 'value_in_rub': -540857,
 'buy_sum_total': 42890,
 'percent': -0.1727442294241082,
 'percent_display': '-17.27%'}

In [76]:
weight_avg_portfolio_plot = make_candle(count_mean_measure_all_sec(dict_companies_data))
plotly.offline.iplot(fig, filename='filename.png')

In [75]:
fig_one_company = make_candle(take_data('tndm'))
plotly.offline.iplot(fig, filename='filename.png')